# Purple Air

### Don't run this again

- response.json contains all the sensors in the purple air database, we filter it to get indexes of sensors in the Greater Boston Area

In [21]:
import pandas as pd
import geopandas as gpd
from geopy import distance

In [22]:
# Run once to create boston sensors csv

all_sensors = pd.read_json("/Users/dhruvchandwani/Downloads/response.json")

all_sensors.describe

all_sensors[["sensor_index", "name","latitude", "longitude"]] = pd.DataFrame(all_sensors['data'].tolist(), index=all_sensors.index)

all_sensors.head()

boston_sensors = all_sensors.loc[(all_sensors["latitude"]>=42.202941) & (all_sensors["latitude"]<=42.475)\
      & (all_sensors["longitude"]<=-70.909495) & (all_sensors["longitude"]>=-71.271667)]

boston_sensors.to_csv("boston_sensors_purpleair.csv")

In [60]:
# read Climate_Ready_Boston_Social_Vulnerability.geojson

tes = gpd.read_file("../data/csv_files/Climate_Ready_Boston_Social_Vulnerability.geojson")

tes["longitude"] = tes["geometry"].centroid.x
tes["latitude"] = tes["geometry"].centroid.y

boston_dem = pd.DataFrame(tes)

boston_sensors = pd.read_csv("../data/boston_sensors_purpleair.csv")

#filtering for sensors with available data
boston_sensors = boston_sensors.loc[boston_sensors['sensor_index'].isin([3642, 26219, 28273, 29389, 34953, 37663, 39657])]

sensor_data = pd.read_csv("../data/sensor_data.csv").drop(columns=['time_stamp'])
print(sensor_data["pm2.5_atm"])

# find closest sensor to all datapoints in the csv
def find_closest_sensor(row, df):
    min_distance = float('inf')
    closest_sensor = None

    for index, sensor_row in df.iterrows():
        coord1 = (row['latitude'], row['longitude'])
        coord2 = (sensor_row['latitude'], sensor_row['longitude'])
        dist = distance.geodesic(coord1, coord2).miles

        if dist < min_distance:
            min_distance = dist
            closest_sensor = sensor_row['sensor_index']

    return closest_sensor

def add_sensor_data(row, df, field):
    temp = df.loc[df['sensor'] == row['closest_sensor']]
    data = temp.iloc[0][field]
    return data

boston_dem['closest_sensor'] = boston_dem.apply(find_closest_sensor, args=(boston_sensors,), axis=1)
boston_dem['pm2.5_atm'] = boston_dem.apply(add_sensor_data, args=(sensor_data, "pm2.5_atm",), axis=1)
boston_dem['pm10_atm'] = boston_dem.apply(add_sensor_data, args=(sensor_data, "pm10.0_atm",), axis=1)

sensors_required = set(boston_dem['closest_sensor'])

print(len(boston_sensors))

boston_dem.head()

boston_dem.drop(columns=['geometry']).to_csv("boston_dem.csv")

/var/folders/dr/5fwr6gys3_q2mlm3nwj706880000gn/T/ipykernel_71990/425769371.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tes["longitude"] = tes["geometry"].centroid.x
/var/folders/dr/5fwr6gys3_q2mlm3nwj706880000gn/T/ipykernel_71990/425769371.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tes["latitude"] = tes["geometry"].centroid.y


0     8.8075
1     8.2420
2     7.7455
3     8.6140
4     7.6985
5    19.4615
6    12.1045
Name: pm2.5_atm, dtype: float64
7
